# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features. 1.1



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

The dataset collected is related to 17 campaigns that occurred between May 2008 and November 2010, corresponding to a total of 79354 contacts. During these phone campaigns, an attractive long-term deposit application, with good interest rates, was offered. For each contact, a large number of attributes was stored and if there was a success (the target variable). There were 6499 successes (8% success rate).

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd
import numpy as np
import time

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV


from scipy.stats import chi2_contingency
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
Bank_marketing_campaigns_df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

Bank_marketing_campaigns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [3]:
print(Bank_marketing_campaigns_df.isnull().sum())
print(Bank_marketing_campaigns_df.info())

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   mon

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

#### To meet the business objective of improving marketing campaign efficiency and maintaining a similar number of successful outcomes with fewer contacts, the following tasks should be carried out:

1. Clean and preprocess the data to handle missing values, inconsistencies, and categorical variables.
2. Perform feature engineering and selection to identify the most relevant features for modeling.
3. Split the data into training and testing sets to evaluate model performance on unseen data.
4. Select and train appropriate classification models, such as logistic regression, decision trees, or support vector machines.
5. Optimize model hyperparameters using techniques like cross-validation, grid search, or randomized search.
6. Evaluate and compare model performance using relevant metrics to select the best model(s) for deployment.
7. Interpret the selected model(s) to understand important features and refine marketing strategies.
8. Deploy the model(s) in a production environment to support decision-making and optimize marketing campaigns.
9. Monitor and maintain the deployed model(s) to ensure continued performance in line with business objectives.

By performing these tasks, you can work towards enhancing the efficiency of marketing campaigns and achieving a similar number of successful customer subscriptions with reduced contact efforts.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

 Perform statistical tests to determine the association between each categorical feature and the target variable. I haved used the Chi-squared test to assess the independence between a categorical feature and the target variable. A low p-value from the Chi-squared test suggests that the categorical feature and the target variable are dependent, which could make the feature useful for your model.

In [4]:
Bank_marketing_campaigns_df = Bank_marketing_campaigns_df.convert_dtypes()
categorical_features = list(Bank_marketing_campaigns_df.select_dtypes(include=['string']).columns)

print("Categorical features:", categorical_features)


rel_feature_list = []
for feature in categorical_features:
    contingency_table = pd.crosstab(Bank_marketing_campaigns_df[feature], Bank_marketing_campaigns_df['y'])
    chi2, p, dof, ex = chi2_contingency(contingency_table)
    rel_feature_list += [feature] if p >= 0.05 else []
    print(f"{feature} - p-value: {p}")
print("Selected Categorical features:", rel_feature_list)
print(f"Based on data we will drop features: {rel_feature_list} that has no relevnes due to soce grter or equil than 0.05")

Bank_marketing_campaigns_df.drop(rel_feature_list, axis=1, inplace=True)

Categorical features: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'y']
job - p-value: 4.189763287563623e-199
marital - p-value: 2.068014648442211e-26
education - p-value: 3.3051890144025054e-38
default - p-value: 5.161957951391637e-89
housing - p-value: 0.05829447669453452
loan - p-value: 0.5786752870441754
contact - p-value: 1.525985652312996e-189
month - p-value: 0.0
day_of_week - p-value: 2.958482005278532e-05
poutcome - p-value: 0.0
y - p-value: 0.0
Selected Categorical features: ['housing', 'loan']
Based on data we will drop features: ['housing', 'loan'] that has no relevnes due to soce grter or equil than 0.05


In [5]:
df = Bank_marketing_campaigns_df
categorical_features = list(Bank_marketing_campaigns_df.columns)
print("Categorical features:", categorical_features)
# for feature in categorical_features:
#     plt.figure(figsize=(10, 5))
#     sns.countplot(data=df, x=feature, hue='y')
#     plt.title(f'{feature} vs Target= Yes/No')
#     plt.show()
#     print("Selected feature name:", feature)

Categorical features: ['age', 'job', 'marital', 'education', 'default', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']


Based on the visualization of relationships between each feature and the target variable, the plots reveal that the following columns may not have a significant effect on the machine learning model:
day_of_week and pdays,

we will drop them to increase processing te performance


In [6]:
Bank_marketing_campaigns_df.drop(['pdays','day_of_week'], axis=1, inplace=True)
categorical_features = list(Bank_marketing_campaigns_df.select_dtypes(include=['string']).columns)
print("Categorical features:", categorical_features)

Categorical features: ['job', 'marital', 'education', 'default', 'contact', 'month', 'poutcome', 'y']


In [7]:
# for feature_1 in categorical_features:
#     for feature_2 in categorical_features:
#         if feature_1 != feature_2:
#             # Create a crosstab of the two categorical features
#             crosstab = pd.crosstab(Bank_marketing_campaigns_df[feature_1], Bank_marketing_campaigns_df[feature_2])

#             # Normalize crosstab to show proportions
#             crosstab_norm = crosstab / crosstab.sum().sum()

#             # Create a heatmap
#             plt.figure(figsize=(10, 5))
#             sns.heatmap(crosstab_norm, annot=True, fmt=".2f", cmap="coolwarm")
#             plt.xlabel(feature_2)
#             plt.ylabel(feature_1)
#             plt.title(f"{feature_1} vs {feature_2}")
#             plt.show()

In [8]:
df = Bank_marketing_campaigns_df
# Encode categorical features using LabelEncoder
le = LabelEncoder()
data_encoded = df.copy()
for feature in categorical_features:
    data_encoded[feature] = le.fit_transform(df[feature])

# Separate features and target variable
X = data_encoded.drop('y', axis=1)
y = data_encoded['y']

# Train a random forest classifier
clf = RandomForestClassifier()
clf.fit(X, y)

# Display feature importances
importances = pd.DataFrame({'feature': X.columns, 'importance': clf.feature_importances_})
importances = importances.sort_values('importance', ascending=False)
print(importances)


           feature  importance
7         duration    0.350790
14       euribor3m    0.129973
0              age    0.107243
15     nr.employed    0.072720
1              job    0.053606
8         campaign    0.048337
3        education    0.047988
10        poutcome    0.041034
13   cons.conf.idx    0.027781
2          marital    0.026105
12  cons.price.idx    0.021795
11    emp.var.rate    0.017728
6            month    0.017546
9         previous    0.016641
5          contact    0.010864
4          default    0.009850


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [9]:


# Separate features and target variable
X = data_encoded.drop('y', axis=1)
y = data_encoded['y']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize or normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [10]:

# value_counts = y_train.value_counts()
# print((100*value_counts[1])/(value_counts[0]+ value_counts[1])  )
# print((100*value_counts[0])/(value_counts[0]+ value_counts[1])  )

# Determine the majority class in the training dataset
majority_class = y_train.value_counts().idxmax()

# Predict the majority class for all samples in the test dataset
y_pred_baseline = [majority_class] * len(y_test)

# Calculate the accuracy of the baseline classifier
baseline_accuracy = accuracy_score(y_test, y_pred_baseline)

print(f"Baseline Accuracy (Majority Class Classifier  where customer will not excpet the offer is): {baseline_accuracy:.4f}")

print(f"Baseline Accuracy (Minority Class Classifier  where customer will  excpet the offer is): {1 - baseline_accuracy:.4f}")


Baseline Accuracy (Majority Class Classifier  where customer will not excpet the offer is): 0.8876
Baseline Accuracy (Minority Class Classifier  where customer will  excpet the offer is): 0.1124


Based on our test data 
Baseline Accuracy (Majority Class Classifier  where customer will not excpet the offer is): 0.8876
Baseline Accuracy (Minority Class Classifier  where customer will  excpet the offer is): 0.1124


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [11]:

logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)
y_pred = logistic_regression.predict(X_test)

In [12]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, zero_division=1)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9102
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     10968
           1       0.67      0.40      0.50      1389

    accuracy                           0.91     12357
   macro avg       0.80      0.69      0.73     12357
weighted avg       0.90      0.91      0.90     12357

Confusion Matrix:
[[10690   278]
 [  832   557]]


### Problem 9: Score the Model

What is the accuracy of our logistic_regression model?

The accuracy of a model is a performance metric that measures the proportion of correct predictions made by the model out of the total number of predictions. In other words, it is the ratio of the number of correct predictions to the total number of predictions made. Accuracy is commonly used to evaluate the performance of classification models.

Accuracy: 0.9102
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     10968
           1       0.67      0.40      0.50      1389

    accuracy                           0.91     12357
   macro avg       0.80      0.69      0.73     12357
weighted avg       0.90      0.91      0.90     12357

Confusion Matrix:
[[10690   278] [  832   557]]


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [13]:
logistic_regression = LogisticRegression()
knn = KNeighborsClassifier()
decision_tree = DecisionTreeClassifier()
svm = SVC()

In [14]:
models = [
    ("Logistic Regression", logistic_regression),
    ("KNN", knn),
    ("Decision Tree", decision_tree),
    ("SVM", svm),
]

In [15]:
results = []

for name, model in models:
    start_time = time.time()
    model.fit(X_train, y_train)
    Train_Time = time.time() - start_time

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    y_train_pred = model.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)

    y_test_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    results.append((name, Train_Time, accuracy, train_accuracy,test_accuracy))

In [16]:
results_df = pd.DataFrame(results, columns=["Model","Train Time","Accuracy","Train Accuracy","Test Accuracy"])
results_df = results_df.sort_values('Accuracy', ascending=False)
print(results_df)

                 Model  Train Time  Accuracy  Train Accuracy  Test Accuracy
3                  SVM    5.860330  0.911791        0.918248       0.911791
0  Logistic Regression    0.037034  0.910172        0.909091       0.910172
1                  KNN    0.002001  0.900461        0.930041       0.900461
2        Decision Tree    0.102093  0.888241        1.000000       0.888241


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [17]:
logistic_regression_params = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

knn_params = {
    'n_neighbors': range(1, 31),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

decision_tree_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 11),
    'min_samples_split': range(2, 21),
    'min_samples_leaf': range(1, 21)
}

svm_params = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

In [18]:
models = [
    ("Logistic Regression", logistic_regression, logistic_regression_params),
    ("KNN", knn, knn_params),
    ("Decision Tree", decision_tree, decision_tree_params),
    ("SVM", svm, svm_params),
]

# models1 = [
#     ("Logistic Regression", logistic_regression, logistic_regression_params),
#     ("KNN", knn, knn_params),
#     ("Decision Tree", decision_tree, decision_tree_params),
    
# ]

In [19]:
tuned_results = []

for name, model, params in models:
    
    grid_search = GridSearchCV(estimator=model, param_grid=params, cv=5, scoring='accuracy', n_jobs=-1)
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    Train_Time = time.time() - start_time

    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    tuned_results.append((model, Train_Time, best_params, best_score))
   

In [ ]:
print(tuned_results)
tuned_results_df = pd.DataFrame(tuned_results, columns=["Model", "Tranning time", "Best_Parameters", "Best Score"])
tuned_results_df = tuned_results_df.astype({'Best_Parameters':'string'})
print(tuned_results_df)

print(tuned_results_df.Best_Parameters)


[(LogisticRegression(), 8.449693441390991, {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}, 0.9092644160945145), (KNeighborsClassifier(), 80.50154304504395, {'metric': 'euclidean', 'n_neighbors': 30, 'weights': 'distance'}, 0.9071139927371524), (DecisionTreeClassifier(), 159.27241444587708, {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 3}, 0.9139123342283633), (SVC(), 646.305481672287, {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}, 0.9093339867574555)]
                      Model  Tranning time  \
0      LogisticRegression()       8.449693   
1    KNeighborsClassifier()      80.501543   
2  DecisionTreeClassifier()     159.272414   
3                     SVC()     646.305482   

                                     Best_Parameters  Best Score  
0  {'C': 0.1, 'penalty': 'l2', 'solver': 'libline...    0.909264  
1  {'metric': 'euclidean', 'n_neighbors': 30, 'we...    0.907114  
2  {'criterion': 'gini', 'max_depth': 5, 'min_sam...    0.913912  
3     

In [ ]:
logistic_regression_params = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
     
}

knn_params = {
    'n_neighbors': range(1, 31),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
    
}

decision_tree_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 11),
    'min_samples_split': range(2, 21),
    'min_samples_leaf': range(1, 21)
    
}

svm_params = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
    
}

In [ ]:
models = [
    ("Logistic Regression", logistic_regression, logistic_regression_params),
    ("KNN", knn, knn_params),
    ("Decision Tree", decision_tree, decision_tree_params),
    ("SVM", svm, svm_params),
]

In [ ]:
tuned_results_randomized = []

n_iter_pram = [12,50,50,16]
Index  = 0

for name, model, params in models:
    
    print(n_iter_pram[Index])
    randomized_search = RandomizedSearchCV(estimator=model, param_distributions=params, n_iter = n_iter_pram[Index], cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
    Index = Index + 1
    start_time = time.time()
    randomized_search.fit(X_train, y_train)
    Train_Time = time.time() - start_time
    

    best_params = randomized_search.best_params_
    best_score = randomized_search.best_score_
  
    tuned_results_randomized.append((model,Train_Time, best_params, best_score))

12
50
50
16


In [ ]:
print(tuned_results_randomized)
print(tuned_results)

tuned_results_randomized_df = pd.DataFrame(tuned_results_randomized, columns=["Model", "Train_Time","Best Parameters", "Best Score"])
print(tuned_results_randomized_df)

[(LogisticRegression(), 12.304203510284424, {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.1}, 0.9092644160945145), (KNeighborsClassifier(), 33.20220994949341, {'weights': 'distance', 'n_neighbors': 28, 'metric': 'euclidean'}, 0.9068018479921613), (DecisionTreeClassifier(), 1.506371259689331, {'min_samples_split': 15, 'min_samples_leaf': 9, 'max_depth': 5, 'criterion': 'gini'}, 0.9137735898648529), (SVC(), 684.0022418498993, {'kernel': 'rbf', 'gamma': 'scale', 'C': 10}, 0.9093339867574555)]
[(LogisticRegression(), 8.449693441390991, {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}, 0.9092644160945145), (KNeighborsClassifier(), 80.50154304504395, {'metric': 'euclidean', 'n_neighbors': 30, 'weights': 'distance'}, 0.9071139927371524), (DecisionTreeClassifier(), 159.27241444587708, {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 3}, 0.9139123342283633), (SVC(), 646.305481672287, {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}, 0.9093339867574555)]
        

##### Questions

create a function that implements the selected model and returns the target value

1. Train the chosen model with the optimal hyperparameters on the entire dataset.

In [ ]:
import numpy as np
import pandas as pd
import ast
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

BMC_test_df =  pd.read_csv('data/bank-additional.csv', sep = ';')
BMC_test_df = BMC_test_df.convert_dtypes()


BMC_test_df.drop(['pdays','day_of_week','housing', 'loan'], axis=1, inplace=True)




In [ ]:
df = BMC_test_df.copy()
# Encode categorical features using LabelEncoder
le = LabelEncoder()
data_encoded = df.copy()
for feature in categorical_features:
    data_encoded[feature] = le.fit_transform(df[feature])


X = data_encoded.drop('y', axis=1)
y = data_encoded['y']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



logistic_regression = LogisticRegression()
knn = KNeighborsClassifier()
decision_tree = DecisionTreeClassifier()
svm = SVC()

logistic_regression_params = {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
knn_params = {'metric': 'euclidean', 'n_neighbors': 30, 'weights': 'distance'}
decision_tree_params = {'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 3}
svm_params =  {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}


# Standardize or normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

LR_best_model = LogisticRegression(**logistic_regression_params)
LR_best_model.fit(X_train, y_train)

y_pred = LR_best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"LR Model accuracy: {accuracy}")


KNN_best_model = KNeighborsClassifier(**knn_params)
KNN_best_model.fit(X_train, y_train)

y_pred = KNN_best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"KNN Model accuracy: {accuracy}")

DT_best_model = DecisionTreeClassifier(**decision_tree_params)
DT_best_model.fit(X_train, y_train)

y_pred = DT_best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"DT Model accuracy: {accuracy}")


SVM_best_model = SVC(**svm_params)
SVM_best_model.fit(X_train, y_train)

y_pred = SVM_best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"SVM Model accuracy: {accuracy}")

best_model = LR_best_model


# def predict_target_value(input_features):
#     reprocessed_features = preprocess(input_features)
#     predicted_target = best_model.predict(preprocessed_features)
#     return predicted_target


LR Model accuracy: 0.9077669902912622
KNN Model accuracy: 0.9077669902912622
DT Model accuracy: 0.8972491909385113
SVM Model accuracy: 0.9053398058252428


In [ ]:
SVM_best_model = SVC(**svm_params)
SVM_best_model.fit(X_train, y_train)
print(X_test[0:2])
y_pred = SVM_best_model.predict(X_train)
print(y_pred)
accuracy = accuracy_score(y_train, y_pred)
print(f"SVM Model accuracy: {accuracy}")

[[ 0.76531368 -1.06479623 -1.91958888  1.04372839 -0.48771952 -0.73825204
  -1.4224708   1.16164424 -0.6055669  -0.35930381  0.19491421 -1.12287967
   0.77642508  0.46152546 -1.55627235 -2.34755585]
 [-0.49454451  0.88005423 -0.28467758 -0.82322923 -0.48771952  1.35455094
  -0.13041557 -0.61166426 -0.6055669  -0.35930381  0.19491421  0.84691492
   1.5283521  -0.29613286  0.77606689  0.84081032]]
[0 0 1 ... 0 0 0]
SVM Model accuracy: 0.9628858827610128
